# Segmenting Toronto Neighborhoods

This notebook retrieves neighborhood information from Wikipedia for Toronto using BeautifulSoup.  It then performs an explore on each neighborhood in Foursquare to retrieve information about venues.  Finally it performs K-means clustering to determine segementation of the neighborhoods.

Import all the libraries needed:

In [38]:
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
#URl lib
import urllib
#geocoder
!pip install geocoder

     |████████████████████████████████| 98 kB 9.3 MB/s  eta 0:00:01


Load the beautifulsoup package:

In [3]:
!pip install beautifulsoup4

from bs4 import BeautifulSoup

Now scrape the Wikipedia page for the table containing the postal codes and create dataframe:

In [43]:
# create columns for dataframe
cols=['PostalCode', 'Borough', 'Neighborhood']

# create empty dataframe
Toronto_df=pd.DataFrame(columns=cols)

#Read the webpage into BS
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

webpage = urllib.request.urlopen(url).read()
page_soup=BeautifulSoup(webpage)

postaltable = page_soup.find("table", {"class": "wikitable sortable"})

n=1

#loop through all the table rows and append data to Toronto_df dataframe
for tr in postaltable.find_all('tr'):

    #skip the header row
    if n==1:
        n=n + 1
        continue
    
    else:
        #ignore not assigned Boroughs
        if tr.find_all('td')[1].text.rstrip('\n') != 'Not assigned':
            # check to see if the neighborhood is not assigned, if so then assign the borough name to the neighborhood
            if tr.find_all('td')[2].text.rstrip('\n') != 'Not assigned':
                
                Toronto_df=Toronto_df.append({
                'PostalCode': tr.find_all('td')[0].text.rstrip('\n'),
                'Borough': tr.find_all('td')[1].text.rstrip('\n'),
                'Neighborhood': tr.find_all('td')[2].text.rstrip('\n'),
            }, ignore_index=True)   
            else:
                Toronto_df=Toronto_df.append({
                'PostalCode': tr.find_all('td')[0].text.rstrip('\n'),
                'Borough': tr.find_all('td')[1].text.rstrip('\n'),
                'Neighborhood': tr.find_all('td')[1].text.rstrip('\n'),
            }, ignore_index=True)   
                    
    
print("Shape: ", Toronto_df.shape)
Toronto_df.head()

Shape:  (103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Now go get the longitude and latitude for each neighborhood

In [ ]:
#import geocoder # import geocoder

#cycle through dataframe to retrieve long/lat for each postal code
#for code in Toronto_df['PostalCode']:
#    # initialize your variable to None
#    lat_lng_coords=None
    
    #loop until we get the coordinates
#    while(lat_lng_coords is None):
#        g=geocoder.google('{}, Toronto, Ontario'.format(code))
#        lat_lng_coords = g.latlng
        
#    Toronto_df['Latitude'] = lat_lng_coords[0]
#    Toronto_df['Longitude'] = lat_lng_coords[1]

#Toronto_df.head()

In [41]:

import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_9a4900c51b86461e8e56e332fd174846 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='uoCEiEiC71U7iWlnO9Yhx3OrQzsv9uLHHpKCHhC9xRJ5',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_9a4900c51b86461e8e56e332fd174846.get_object(Bucket='capstoneproject-donotdelete-pr-g6gysetnvfvfbo',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

postalcodelatlong = pd.read_csv(body)
postalcodelatlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
#merge postalcodelatlong with Toronto_df
Toronto_df_merged=Toronto_df
Toronto_df_merged=Toronto_df_merged.join(postalcodelatlong.set_index('Postal Code'), on='PostalCode' )
Toronto_df_merged.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [112]:
Toronto_df_merged.head(60)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Get setup to retreive Foursquare data by including some additional libraries and initializations

In [45]:
#import additional libraries needed
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#get lat, long for Toronto
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

#Foursquare credentials
CLIENT_ID = 'removed for security' # your Foursquare ID
CLIENT_SECRET = 'removed for security' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value



Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.11.0             |   py37h89c1867_0         168 KB  conda-forge
    aiohttp-3.7.3              |   py37h5e8e339_2         629 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
   

libnghttp2-1.43.0    | 808 KB    | ##################################### | 100% 
xorg-libxdmcp-1.1.3  | 19 KB     | ##################################### | 100% 
idna-2.10            | 52 KB     | ##################################### | 100% 
qt-5.12.9            | 99.5 MB   | ##################################### | 100% 
pytz-2021.1          | 239 KB    | ##################################### | 100% 
nbformat-5.1.2       | 66 KB     | ##################################### | 100% 
libtiff-4.2.0        | 633 KB    | ##################################### | 100% 
json5-0.9.5          | 20 KB     | ##################################### | 100% 
astropy-4.2          | 7.5 MB    | ##################################### | 100% 
cycler-0.10.0        | 9 KB      | ##################################### | 100% 
krb5-1.17.2          | 1.4 MB    | ##################################### | 100% 
brotlipy-0.7.0       | 341 KB    | ##################################### | 100% 
libaec-1.0.4         | 31 KB

async_generator-1.10 | 18 KB     | ##################################### | 100% 
readline-8.0         | 281 KB    | ##################################### | 100% 
brunsli-0.1          | 200 KB    | ##################################### | 100% 
retrying-1.3.3       | 11 KB     | ##################################### | 100% 
pytest-6.2.2         | 430 KB    | ##################################### | 100% 
tensorflow-estimator | 645 KB    | ##################################### | 100% 
gmpy2-2.1.0b1        | 206 KB    | ##################################### | 100% 
python-3.7.10        | 57.3 MB   | ##################################### | 100% 
secretstorage-3.3.1  | 24 KB     | ##################################### | 100% 
olefile-0.46         | 32 KB     | ##################################### | 100% 
ibm-wsrt-py37main-ma | 2 KB      | ##################################### | 100% 
cytoolz-0.11.0       | 403 KB    | ##################################### | 100% 
jedi-0.18.0          | 923 K

py-1.10.0            | 73 KB     | ##################################### | 100% 
scipy-1.5.3          | 18.5 MB   | ##################################### | 100% 
tensorflow-base-1.14 | 87.6 MB   | ##################################### | 100% 
pyqtchart-5.12       | 256 KB    | ##################################### | 100% 
lxml-4.6.2           | 1.5 MB    | ##################################### | 100% 
pthread-stubs-0.4    | 5 KB      | ##################################### | 100% 
pyzmq-22.0.3         | 526 KB    | ##################################### | 100% 
joblib-1.0.1         | 206 KB    | ##################################### | 100% 
setuptools-49.6.0    | 947 KB    | ##################################### | 100% 
cryptography-3.4.4   | 636 KB    | ##################################### | 100% 
oauthlib-3.0.1       | 82 KB     | ##################################### | 100% 
matplotlib-3.3.4     | 6 KB      | ##################################### | 100% 
pyparsing-2.4.7      | 60 KB

done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          98 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0



geopy-2.1.0          | 64 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction:

In [162]:
print(CLIENT_ID)
lat=43.667856
#lat=43.650571
lng=-79.532242
#lng=-79.384568
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
# make the GET request
results = requests.get(url).json()["response"]['groups'][0]['items']
if results:
    print("Got some")
    results
else:
    print("empty")

1FGMUP2IQT0WWIISY0XIIF5AYR4NDHHTBAOTMOFAP2YYI2DB
empty


In [163]:
#function to return nearby venues for a particular location
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #some requests are coming back empty so check to make sure we got results
        if results:
            
            #return only relevant information for each nearby venue
            venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
            
        else:
            print("no results for: ", name)
            
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Go get the nearby venue data for all locations

In [164]:
#get all venues
Toronto_venues=getNearbyVenues(Toronto_df_merged['Neighborhood'], Toronto_df_merged['Latitude'], Toronto_df_merged['Longitude'])

print("Shape: ", Toronto_venues.shape)
Toronto_venues.head()

Toronto_venues.groupby('Neighborhood').count

print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))



Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
no results for:  Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Lit

In [156]:
#analyze each neighborhood
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#one of the venue categories is called neighborhood so go ahead and drop that one
Toronto_onehot.drop(['Neighborhood'], axis=1, inplace=True)

# add neighborhood column back to dataframe on the first column
Toronto_onehot.insert(0, "Neighborhood", Toronto_venues['Neighborhood'], True) 

Toronto_onehot.head()


,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now groupby neighborhood and generate the mean for each category to be used in the K-means

In [157]:
#create new df with mean of categories by neighborhood
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()

print("Shape: ", Toronto_grouped.shape)
Toronto_grouped


Shape:  (96, 269)


,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
#function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


Generate a dataframe containing the top 10 venues in each neighborhood

In [166]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Coffee Shop,Athletics & Sports,Pub,Sandwich Place,Distribution Center,Dim Sum Restaurant,Diner
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Supermarket,Sushi Restaurant,Intersection,Shopping Mall,Middle Eastern Restaurant,Restaurant,Mobile Phone Shop
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Comfort Food Restaurant,Thai Restaurant,Liquor Store,Juice Bar,Fast Food Restaurant,Butcher,Restaurant


Now run K-means to cluster neighborhoods into 5 different clusters

In [167]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Now create a new dataframe merging cluster information with top venue information for each neighborhood

In [168]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_df_merged

# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='right')

Toronto_merged.head() 


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Portuguese Restaurant,Intersection,Pizza Place,Coffee Shop,Financial or Legal Service,Hockey Arena,Donut Shop,Diner,Discount Store,Distribution Center
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Theater,Pub,Breakfast Spot,Café,Electronics Store,Restaurant,Performing Arts Venue
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Gift Shop,Event Space,Furniture / Home Store,Coffee Shop,Accessories Store,Vietnamese Restaurant,Boutique,Miscellaneous Shop,Eastern European Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place,Café


Now create a folio map of clusters

In [170]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
#had trouble with trying to include different colors so I just commented that out

Now examine clusters to determine any observations

In [176]:
#Cluster 1
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
21,Caledonia-Fairbanks,Park,Pool,Women's Store,Gourmet Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
32,Scarborough Village,Pizza Place,Playground,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
49,"North Park, Maple Leaf Park, Upwood Park",Park,Basketball Court,Bakery,Construction & Landscaping,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
50,Humber Summit,Furniture / Home Store,Intersection,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio,Dessert Shop
61,Lawrence Park,Park,Bus Line,Swim School,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Dim Sum Restaurant
68,"Forest Hill North & West, Forest Hill Road Park",Park,Jewelry Store,Bus Line,Sushi Restaurant,Trail,Dumpling Restaurant,Drugstore,Eastern European Restaurant,Donut Shop,Dessert Shop
77,"Kingsview Village, St. Phillips, Martin Grove ...",Mobile Phone Shop,Bus Line,Sandwich Place,Park,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
85,"Milliken, Agincourt North, Steeles East, L'Amo...",Park,Intersection,Playground,Coffee Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
91,Rosedale,Park,Playground,Trail,Escape Room,Electronics Store,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Department Store


In [177]:
#Cluster 2
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,Portuguese Restaurant,Intersection,Pizza Place,Coffee Shop,Financial or Legal Service,Hockey Arena,Donut Shop,Diner,Discount Store,Distribution Center
2,"Regent Park, Harbourfront",Coffee Shop,Bakery,Park,Theater,Pub,Breakfast Spot,Café,Electronics Store,Restaurant,Performing Arts Venue
3,"Lawrence Manor, Lawrence Heights",Clothing Store,Gift Shop,Event Space,Furniture / Home Store,Coffee Shop,Accessories Store,Vietnamese Restaurant,Boutique,Miscellaneous Shop,Eastern European Restaurant
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place,Café
6,"Malvern, Rouge",Fast Food Restaurant,Dessert Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
...,...,...,...,...,...,...,...,...,...,...,...
97,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,American Restaurant,Salad Place,Steakhouse,Asian Restaurant
98,"The Kingsway, Montgomery Road, Old Mill North",River,Pool,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
99,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Pub,Hotel,Café,Burger Joint
100,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Smoke Shop,Auto Workshop,Brewery,Burrito Place,Comic Shop,Farmers Market,Fast Food Restaurant,Garden


In [178]:
#Cluster 3
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,"Humberlea, Emery",Baseball Field,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant
101,"Old Mill South, King's Mill Park, Sunnylea, Hu...",Baseball Field,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant


In [179]:
#Cluster 4
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,"East Toronto, Broadview North (Old East York)",Convenience Store,Park,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
64,Weston,Convenience Store,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
66,York Mills West,Convenience Store,Park,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio


In [180]:
#Cluster 5
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"West Deane Park, Princess Gardens, Martin Grov...",Bakery,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant


Final observations:
Cluster 1 - seems to be highly related to outdoor activities
Cluster 2 - seems to be focussed on coffee shops and restaurants
Cluster 3 - seems to be related to athletic activities
Cluster 4 - seems to be related to stores
Cluster 5 - no observation